# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../WeatherPy/output_data/cities.csv')
#df = df.head(50)
df

,City,Country,Temperature,Humidity,Cloudiness,Wind speed,Latitude,Longitude,Max Temperature,Date
0,talavera,PH,80.60,74,75,22.37,15.6167,120.9333,80.60,1613185555
1,saskylakh,RU,-27.94,82,77,1.92,71.9167,114.0833,-27.94,1613185555
2,geraldton,AU,91.40,35,75,23.02,-28.7667,114.6000,91.40,1613185555
3,vaini,TO,86.00,74,75,8.05,-21.2000,-175.2000,86.00,1613185556
4,butaritari,KI,81.32,83,0,12.15,3.0707,172.7902,81.32,1613185279
...,...,...,...,...,...,...,...,...,...,...
545,salinas,US,53.08,82,40,5.75,36.6777,-121.6555,53.60,1613185707
546,uni,RU,10.09,96,100,9.10,57.7535,51.4923,10.09,1613185707
547,sarangani,PH,82.17,80,81,13.15,5.4033,125.4636,82.17,1613185707
548,flinders,AU,64.99,95,100,1.01,-34.5833,150.8552,64.99,1613185708


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(df[["Latitude", "Longitude"]], weights=df['Humidity'],
                                 max_intensity=df['Humidity'].max(),
                                 dissipating=False,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
df_clean = df.loc[(df["Max Temperature"] > 70) &
                                          (df["Max Temperature"] < 80) &
                                          (df["Wind speed"] < 10) &
                                          (df["Cloudiness"] == 0)]
# Display to output
df_clean

,City,Country,Temperature,Humidity,Cloudiness,Wind speed,Latitude,Longitude,Max Temperature,Date
56,busselton,AU,75.00,58,0,1.99,-33.6500,115.3333,75.00,1613185572
150,vila velha,BR,76.48,83,0,5.75,-20.3297,-40.2925,77.00,1613185600
160,san lorenzo,PY,73.40,88,0,4.29,-25.3333,-57.5333,73.40,1613185603
239,pisco,PE,72.39,73,0,8.05,-13.7000,-76.2167,73.40,1613185473
253,neuquen,AR,77.00,50,0,2.30,-38.9516,-68.0591,77.00,1613185627
281,pedernales,DO,76.51,62,0,8.23,18.0384,-71.7440,76.51,1613185635
284,dabakala,CI,76.03,54,0,4.36,8.3667,-4.4333,76.03,1613185636
300,umm lajj,SA,73.20,60,0,9.93,25.0213,37.2685,73.20,1613185640
335,mawlaik,MM,76.66,36,0,2.19,23.6333,94.4167,76.66,1613185649
345,harper,LR,78.93,89,0,6.96,4.3750,-7.7169,78.93,1613185652


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = df_clean
hotel_df['Hotel Name'] = ''
hotel_df.head()

C:\Users\HPS\anaconda3\envs\pythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Temperature,Humidity,Cloudiness,Wind speed,Latitude,Longitude,Max Temperature,Date,Hotel Name
56,busselton,AU,75.00,58,0,1.99,-33.6500,115.3333,75.0,1613185572,
150,vila velha,BR,76.48,83,0,5.75,-20.3297,-40.2925,77.0,1613185600,
160,san lorenzo,PY,73.40,88,0,4.29,-25.3333,-57.5333,73.4,1613185603,
239,pisco,PE,72.39,73,0,8.05,-13.7000,-76.2167,73.4,1613185473,
253,neuquen,AR,77.00,50,0,2.30,-38.9516,-68.0591,77.0,1613185627,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    params["location"] = f"{row['Latitude']},{row['Longitude']}"    
    response = requests.get(base_url, params=params).json()
    hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]

C:\Users\HPS\anaconda3\envs\pythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))